# Acquire

In [1]:
# import acquire as a
# from env import github_token, github_username
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import scipy.stats as stats
from scipy.stats import  chi2_contingency
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

import sys
sys.path.append("..")
import acquire as a
from env import github_token, github_username
"""
A module for obtaining repo readme and language data from the github API.
Before using this module, read through it, and follow the instructions marked
TODO.
After doing so, run it like this:
    python acquire.py
To create the `data.json` file that contains the data.
"""
import json
from typing import Dict, List, Optional, Union, cast



## Get data python

In [5]:
# repo = 'python/cpython' # repository identification
# authentications
headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

REPO_NAME = []
for i in range(1, 10):
    # url to python repos
    python_url= f'https://github.com/search?o=desc&q=stars%3A%3E1+language%3APython&s=forks&type=Repositories&p=1&l=python={i}'

    # get json rescponce
    res = a.github_api_request(python_url)
    # repo_content = a.get_repo_contents(repo)
    temp_name = [REPO_NAME.append(x['hl_name']) for x in res['payload']['results']]


In [6]:
url_link = []
readme_con = []
repo_language = []
rst_file= []
for page_repo in REPO_NAME:
    repo_content = a.get_repo_contents(page_repo)
    # locate the Readme.rst file link
    for ele in range(len(repo_content)):
        link = repo_content[ele]["html_url"]
        match = re.search(f"README", link)
        if match:
            rst_file = link
            break
    # get the readme request
    readme_res = requests.get(rst_file)
    
    if readme_res.status_code == 200:
        # find read me content
        soup = BeautifulSoup(readme_res.text, 'html.parser')
        readme_content = soup.get_text()

        # extract noisy charactures from the content
        pattern = r'"richText":"(.*?)"\s*,\s*"renderedFileInfo"'
        matches = re.findall(pattern, readme_content)
        extracted_content = matches[0].replace("\\n","")
        
        # get repo language
        repo_lang = a.get_repo_language(page_repo)

        # url and readme content of all repo
        url_link.append(rst_file)
        readme_con.append(extracted_content)
        repo_language.append(repo_lang)
    else:
        print("Failed to connect...!")

In [8]:
results = pd.DataFrame(REPO_NAME, columns=["repo_name"]).assign(url = url_link, 
                                                                language = repo_language,
                                                                readme_content = readme_con)
results

,repo_name,url,language,readme_content
0,jackfrued/Python-100-Days,https://github.com/jackfrued/Python-100-Days/b...,Python,Python - 100天从新手到大师作者：骆昊说明：从项目上线到获得8w+星标以来，一直收...
1,Significant-Gravitas/Auto-GPT,https://github.com/Significant-Gravitas/Auto-G...,Python,Auto-GPT: An Autonomous GPT-4 Experiment💡 Get ...
2,hankcs/HanLP,https://github.com/hankcs/HanLP/blob/master/RE...,Python,HanLP: Han Language Processing ...
3,XX-net/XX-Net,https://github.com/XX-net/XX-Net/blob/master/R...,Python,🚀 XX-Net (翻墙VPN)这是一个可靠的翻墙系统，已经连续运行 8 年！我们不去研究墙...
4,littlecodersh/ItChat,https://github.com/littlecodersh/ItChat/blob/m...,Python,itchat English versionitchat是一个开源的微信个人号接口，使用...
...,...,...,...,...
85,pallets/flask,https://github.com/pallets/flask/blob/main/REA...,Python,FlaskFlask is a lightweight WSGI web applicati...
86,Ebazhanov/linkedin-skill-assessments-quizzes,https://github.com/Ebazhanov/linkedin-skill-as...,Python,Linkedin Skill assessments - Answers⚠️ DISCLAI...
87,deepfakes/faceswap,https://github.com/deepfakes/faceswap/blob/mas...,Python,deepfakes_faceswap FaceSwap is a tool that ut...
88,geekcomputers/Python,https://github.com/geekcomputers/Python/blob/m...,Python,My Python Eggs 🐍 😄I do not consider myself as ...


## Get data Java-script

In [15]:
# repo = 'python/cpython' # repository identification
# authentications
headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

REPO_NAME = []
for i in range(1, 10):
    # url to python repos
    python_url= f'https://github.com/search?o=desc&q=stars%3A%3E1+language%3AJavaScript&s=forks&type=Repositories&l=JavaScript&p={i}'

    # get json rescponce
    res = a.github_api_request(python_url)
    # repo_content = a.get_repo_contents(repo)
    temp_name = [REPO_NAME.append(x['hl_name']) for x in res['payload']['results']]

repo_names = []
url_link = []
readme_con = []
repo_language = []
rst_file= []
for page_repo in REPO_NAME:
    repo_content = a.get_repo_contents(page_repo)
    # locate the Readme.rst file link
    for ele in range(len(repo_content)):
        link = repo_content[ele]["html_url"]
        match = re.search(f"README", link)
        if match:
            rst_file = link
            break
    # get the readme request
    readme_res = requests.get(rst_file)
    
    if readme_res.status_code == 200:
        # find read me content
        soup = BeautifulSoup(readme_res.text, 'html.parser')
        readme_content = soup.get_text()

        # extract noisy charactures from the content
        pattern = r'"richText":"(.*?)"\s*,\s*"renderedFileInfo"'
        matches = re.findall(pattern, readme_content)
        if len(matches) == 1:
            extracted_content = matches[0].replace("\\n","")
        
            # get repo language
            repo_lang = a.get_repo_language(page_repo)

            # url and readme content of all repo
            repo_names.append(page_repo)
            url_link.append(rst_file)
            readme_con.append(extracted_content)
            repo_language.append(repo_lang)
    else:
        print("Failed to connect...!")

In [16]:
results = pd.DataFrame(repo_names, columns=["repo_name"]).assign(url = url_link, 
                                                                language = repo_language,
                                                                readme_content = readme_con)
results

,repo_name,url,language,readme_content
0,twbs/bootstrap,https://github.com/twbs/bootstrap/blob/main/RE...,JavaScript,"Bootstrap Sleek, intuitive, and power..."
1,github/docs,https://github.com/github/docs/blob/main/READM...,JavaScript,GitHub Docs This repository contains the docum...
2,facebook/react,https://github.com/facebook/react/blob/main/RE...,JavaScript,React · React is a JavaScript library for b...
3,mrdoob/three.js,https://github.com/mrdoob/three.js/blob/dev/RE...,JavaScript,three.jsJavaScript 3D libraryThe aim of the pr...
4,academicpages/academicpages.github.io,https://github.com/academicpages/academicpages...,JavaScript,A Github Pages template for academic websites....
...,...,...,...,...
83,microsoft/monaco-editor,https://github.com/microsoft/monaco-editor/blo...,JavaScript,Monaco EditorThe Monaco Editor is the fully fe...
84,learn-co-students/javascript-strings-lab-js-in...,https://github.com/learn-co-students/javascrip...,JavaScript,"JavaScript Strings LabOverviewIn this lab, we'..."
85,dropzone/dropzone,https://github.com/dropzone/dropzone/blob/main...,JavaScript,Dropzone is a JavaScript library that turns an...
86,ibm-developer-skills-network/qkfls-Movie-Revie...,https://github.com/ibm-developer-skills-networ...,JavaScript,Movies Review Database
